# Fast imputation module - example job request

## Imports and globals

Here you will have to set you own `apikey` by replacing the `enter-your-api-key-here` text.

In [ ]:
import requests
import json
import sys

# Change apikey with the apikey retrieved from deploit's webpage
apikey = "enter-your-api-key-here"

# This is the url for deploit service. Do not change it!
deploit_url = "https://deploit.lifebit.ai"

## Fetch project id

In order to run any job in Deploit you need to have a project to which you will associate your jobs. Therefore, you need to retrieve the project id for the project name in which you want to associated the job that you are running. For example, you may want to run the job within the `demo` project!

In [ ]:
# This function will fetch the project id for a given project name
def fetch_deploit_project_id(apikey, project_name):
    data = {"apikey": apikey}
    r = requests.get("{}/api/v1/projects".format(deploit_url), params=data)
    for project in json.loads(r.content):
        if project["name"] == project_name:
            return project["_id"]

In [ ]:
project_id = fetch_deploit_project_id(apikey, "Demo")
if not project_id:
    sys.exit("Project id couldn't be find. Check that you provided a valid project name")

## Fetch workflow id

Then let's fetch the workflow id for the Fast imputation module:

In [ ]:
# This function will fetch the workflow id for a given workflow name
def fetch_deploit_workflow_id(apikey, workflow_name):
    data = {"apikey": apikey}
    r = requests.get("{}/api/v1/workflows".format(deploit_url), params=data)
    for workflow in json.loads(r.content):
        if workflow["name"] == workflow_name:
            return workflow["_id"]

In [ ]:
workflow_id = fetch_deploit_workflow_id(apikey, "Fast imputation")
if not workflow_id:
    sys.exit("workflow id couldn't be find. Check that you provided a valid workflow name")

## Running the job and checking it's status

The following snippet takes the ids fetch above and makes a request to deploit api:

In [ ]:
def send_job_request_to_deploit(apikey, workflow_id, project_id):
    # Prepare api request for deploit to run a job
    headers = {
        "Content-type": "application/json",
        "apikey": apikey
    }

    # This parameter contains example files that are available in deploit's featured datasets.
    workflow_params = [
       {
         "name": "genotypes_path",
         "prefix": "--",
         "dataItemEmbedded": {
            "type": "S3File",
            "data": {
              "name": "uk35C650_20170608144013.txt",
              "s3BucketName": "lifebit-featured-datasets",
              "s3ObjectKey": "modules/ancestry/uk35C650_20170608144013.txt"
            }
         }
       }
    ]

    params = {
        "executionPlatform": "aws",
        # Here we are setting the proper on demand instance for the module.
        # Note that smaller instances might not get the job done.
        "instanceType": "m4.2xlarge",
        "parameters": workflow_params,
        "project": project_id,
        "workflow": workflow_id
    }

    r = requests.post("{}/api/v1/jobs".format(deploit_url), data=json.dumps(params), headers=headers)
    return json.loads(r.content)["_id"]

In [ ]:
job_id = send_job_request_to_deploit(apikey, workflow_id, project_id)
# If everything works as expected this should print an address to deploit job page.
if job_id:
    print("Job successfully sent to deploit. You can check the status " +
          "of the job in {}/app/jobs/{}".format(deploit_url, job_id))